In [4]:
acceptable_distance = 10

In [9]:
import numpy as np
from shapely.geometry import Point, LineString


class TheRightRoute():
    def __init__(self, waypoints: LineString, acceptable_distance):
        self.route = waypoints
        self.acceptable_distance = acceptable_distance
        
        self.start_pt = Point(self.route.coords[0])
        self.finish_line = self.calc_finish_line()
    
    def calc_finish_line(self):
        if self.route.coords[-1][0]-self.route.coords[-2][0] == 0:
            RouteEndAngle = np.pi /2
        else:
            RouteEndAngle = np.arctan((self.route.coords[-1][1]-self.route.coords[-2][1])/ \
                                      (self.route.coords[-1][0]-self.route.coords[-2][0]))
        finish_line = LineString([
            Point(
                self.route.coords[-1][0] - acceptable_distance * np.sin(RouteEndAngle),
                self.route.coords[-1][1] + acceptable_distance * np.cos(RouteEndAngle)),
            Point(
                self.route.coords[-1][0] + acceptable_distance * np.sin(RouteEndAngle),
                self.route.coords[-1][1] - acceptable_distance * np.cos(RouteEndAngle))
        ])
        return finish_line
    
    
    
    
    
# RouteToFollow = LineString([(0,0), (60,100)]) # Straight line
RouteToFollow = LineString([(0,0), (60, 0), (60, 60)]) # L-shape
# RouteToFollow = LineString([(0,0), (60, 0), (60, 60), (100, 50), (90, 30), (90, 0), (120,0)]) # Bumpy road
# RouteToFollow = LineString([(0,0), (60, 0), (60, 60), (100, 50), (75, 30), (90, 0), (120,0)]) # Bumpy road with shortcut
# RouteToFollow = LineString([(0,0), (60, 0), (60, 60), (100, 50), (90, 30), (90, 0), (120,0), (130,90), (10, 85), (10, 30)]) # Snail
# RouteToFollow = LineString([(0,0), (20, 0), (20, 30), (50, 30), (50, -30), (20, -30), (20,-60), (80,-60), (80, 60), \
#                             (-30, 60), (-30, 0), (0, -90), (110, -90), (110, 0), (130, 0)]) # Labyrinth
the_right_route = TheRightRoute(RouteToFollow, acceptable_distance)

In [ ]:
import numpy as np

class FinderStrategy_1():
    
    def __init__(self, n_steps, max_steering):
        self.n_steps = n_steps
        self.max_steering = max_steering
    
    
    def generate_random_steering_angles(self, n_trajectories: int):
        return [np.array([0] + list(self.max_steering*np.random.normal(0, 0.5, size=self.n_steps))) \
                for i in range(n_trajectories)]
    
    
    def generate_twins(self, df_vehicles):
        v_best_score = df_vehicles.nlargest(n=1, columns='Score').iloc[0]
        base = v_best_score['Steering angles']
        n_adnl_combinations = n_cars - 1

        max_upto = int(v_best_score['TrajectoryLength'])
        shorten_halflength = int(0.05*v_best_score['TrajectoryLength'])
        list_keep_upto = max_upto - np.clip(np.abs(np.random.normal(0, max_upto/3.3, (n_adnl_combinations))), 0, max_upto).astype(int)

        output = [make_new_branch(base, keep_upto) if keep_upto > int(0.80*max_upto) \
                  else shorten_branch(base, v_best_score['Trajectory'], int(keep_upto-shorten_halflength), int(keep_upto+shorten_halflength)) \
                  for keep_upto in list_keep_upto]
        return [base] + output
    
    
    def calc_score(self, vehicle: pd.Series):
        return vehicle['DistanceFromStart'] - sum([abs(val) for val in vehicle['Steering angles']])

In [10]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from itertools import takewhile
from shapely.geometry import Point, LineString

class PathFinder():
    def __init__(self, the_right_route, finder_strategy, acceptable_distance, n_cars, n_steps, n_loops, step_length):
        self.the_right_route = the_right_route
        self.finder_strategy = finder_strategy
        
        self.acceptable_distance = acceptable_distance
        self.n_cars = n_cars
        self.n_steps = n_steps
        self.n_loops = n_loops
        self.step_length = step_length
    
    
    def run_simulation(self):
        steering_angles = self.finder_strategy.generate_random_steering_angles(self.n_cars, self.n_steps)
        vehicles_history = pd.DataFrame()

        for loop in tqdm(range(self.n_loops+1)):
            vehicles = self.calc_vehicles_df(loop, steering_angles)
            vehicles_history = pd.concat([vehicles_history, vehicles])
            steering_angles = self.finder_strategy.generate_twins(vehicles) # Prep next iteration. MOST IMPORTANT STEP FOR LEARNING!
        return vehicles_history
    
    
    def calc_vehicles_df(self, loop, steering_angles):
        vehicles = pd.DataFrame({'Loop': loop, 'Steering angles': steering_angles})
        vehicles['Trajectory'] = vehicles['Steering angles'].apply(calc_trajectory)
        vehicles['DistanceFromStart'] = vehicles['Trajectory'].apply(calc_distance_from_start)
        vehicles['TrajectoryLength'] = vehicles['Trajectory'].apply(lambda traj: traj.length)
        vehicles['Score'] = vehicles.apply(self.finder_strategy.calc_score, axis=1)
        return vehicles
    
    
    def calc_trajectory(self, steering_angles):
        steering_angles_cumsum = np.cumsum(steering_angles)
        Xpts = (self.step_length*np.cos(steering_angles_cumsum)).cumsum()
        Ypts = (self.step_length*np.sin(steering_angles_cumsum)).cumsum()
        
        trajectory = [self.the_right_route.start_pt] + list(takewhile(lambda pt: pt.distance(self.the_right_route.route) <= self.acceptable_distance, self.gen_points(Xpts, Ypts)))
        return LineString(trajectory)
    
    
    def gen_points(self, Xpts, Ypts):
        for pt in list(zip(Xpts, Ypts)):
            yield Point(pt[0], pt[1])
    
    
    def calc_distance_from_start(self, trajectory):
        return self.the_right_route.route.project(Point(list(trajectory.coords)[-1]))
    
    
    


pathfinder(the_right_route, acceptable_distance)